# Managing Your Data Requests with `ecmwf-datastores-client`

This guide shows you how to track and manage all the climate/atmosphere data requests you've submitted to the Copernicus Data Store. Think of this as your "data request history manager."

## What You Need Before Starting

1. An **active internet connection**
2. **Python** installed on your computer
3. The `ecmwf-datastores-client` package installed, otherwise uncomment and run the next cell
4. A **CDS account** or **ADS account** with your API key set up (see the "Getting Started" notebook)

In [1]:
# !pip install -U ecmwf-datastores-client

In [2]:
import os
from ecmwf.datastores import Client
import pandas as pd
from IPython.display import display


**Tip:** If you don’t want to see warnings while running your notebook, you can uncomment the following cell:

In [3]:
# import warnings
# warnings.filterwarnings("ignore")

## Connect to the Data Store

First, let's establish a connection to the data store:

In [4]:
client = Client(
    url=os.getenv("ECMWF_DATASTORES_URL"),
    key=os.getenv("ECMWF_DATASTORES_KEY"),
)

# Check that we're connected successfully
connection_info = client.check_authentication()
print("✅ Connected successfully to the Data Store!")

[2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
[2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


✅ Connected successfully to the Data Store!


## View Your Data Request History

Let's retrieve information about all your past data requests:

In [5]:
# Get a list of your data requests, newest first
print("Retrieving your data request history...")
jobs = client.get_jobs(sortby="-created")

# Create a list to store all the request IDs and their status
request_ids = []

# Loop through all pages of results (if there are many requests)
while jobs is not None:
    # Add each job's ID and status to our list
    request_ids.extend([job['jobID'], job['status']] for job in jobs.json['jobs'])
    # Move to the next page of results
    jobs = jobs.next

print(f"Found {len(request_ids)} data requests in your history!")

Retrieving your data request history...
Found 35 data requests in your history!


## Get Detailed Information About Your Requests

Now let's collect more detailed information about each of your data requests:

In [6]:
print("Gathering detailed information about each request...")
jobs_info = []

# Loop through each request and collect its details
for request_id, request_status in request_ids:
    # Get the full details of this particular request
    remote = client.get_remote(request_id)

    # Get date components
    year = ", ".join(filter(None, remote.request.get("year", [None])))
    month = ", ".join(filter(None, remote.request.get("month", [None])))
    day = ", ".join(filter(None, remote.request.get("day", [None])))
    
    # Combine date information
    date_info = f"{year}-{month}-{day}" if all([year, month, day]) else "Not specified"
    
    # Extract and organize the important information
    jobs_info.append({
        "Request ID": request_id,
        "Status": request_status,
        "Dataset": remote.collection_id,
        "Created": remote.created_at.strftime("%Y-%m-%d %H:%M"),
        "Completed": remote.finished_at.strftime("%Y-%m-%d %H:%M") if remote.finished_at else "Not finished",
        "Variable": ", ".join(filter(None, remote.request.get("variable", [None]))),
        "Date": date_info,
        "Format": remote.request.get("data_format", "Unknown")
    })

print("✅ Information collected successfully!")

Gathering detailed information about each request...
✅ Information collected successfully!


## Create an Interactive Table of Your Requests

Now let's display all your data requests in an easy-to-use interactive table:

In [7]:
# Create a table from our collected information
df_jobs = pd.DataFrame(jobs_info)

# Render DataFrame as HTML to avoid truncation
display(df_jobs)

,Request ID,Status,Dataset,Created,Completed,Variable,Date,Format
0,acd17ffb-1be0-4e10-9e51-a28dd8d1b4ac,successful,reanalysis-era5-single-levels,2025-05-16 10:54,2025-05-16 10:55,2m_temperature,2022-01-01,grib
1,0eca863a-8146-41ef-914a-ee8c4264ae0e,successful,reanalysis-era5-single-levels,2025-05-16 10:53,2025-05-16 10:54,2m_temperature,2022-01-01,grib
2,abef69ef-68d1-4895-bcde-1f633e900436,successful,reanalysis-era5-single-levels,2025-05-16 10:53,2025-05-16 10:53,2m_temperature,2022-01-01,grib
3,47c7a4ef-efd6-48a4-a142-8876738e34bc,successful,reanalysis-era5-single-levels,2025-05-16 10:52,2025-05-16 10:52,2m_temperature,2022-01-01,grib
4,137f2523-225c-403a-bf5c-9b559aa74dfa,successful,reanalysis-era5-single-levels,2025-05-16 10:51,2025-05-16 10:51,2m_temperature,2022-01-01,grib
5,b10772ba-6d06-4d6f-ad26-e5c9eb5915e4,successful,reanalysis-era5-single-levels,2025-05-16 10:51,2025-05-16 10:51,2m_temperature,2022-01-01,grib
6,ab04ec27-e9a0-47b5-b15a-21f441208f03,successful,reanalysis-era5-single-levels,2025-05-16 10:49,2025-05-16 10:50,2m_temperature,2022-01-01,grib
7,a47fb779-558d-4453-b586-e2fc7941c1d3,successful,reanalysis-era5-single-levels,2025-05-16 10:49,2025-05-16 10:49,2m_temperature,2022-01-01,grib
8,1c3ebbde-e2d3-4311-855f-563db8c06013,successful,reanalysis-era5-single-levels,2025-05-16 10:49,2025-05-16 10:49,2m_temperature,2022-01-01,grib
9,3b525e6e-9aab-46ee-9e4c-2e43e1a4f6f0,successful,reanalysis-era5-single-levels,2025-05-16 10:48,2025-05-16 10:48,2m_temperature,2022-01-01,grib


In [8]:
df_jobs['Request ID'][0]

'acd17ffb-1be0-4e10-9e51-a28dd8d1b4ac'

## Working with Your Past Data Requests

Here are some examples of what you can do with your past requests:

### Download Data from a Past Request

In [9]:
# If you want to download data from a specific request:
request_id = "49b3a61b-7310-4da3-b22f-f9e85f2a9821"  # Replace with an ID from your table

# Check if the request was successful
remote = client.get_remote(request_id)
if remote.status == "successful":
    print(f"Downloading data from request made on {remote.created_at.strftime('%Y-%m-%d')}...")
    remote.download(target="my_past_request_data.grib")
    print("✅ Download complete!")
else:
    print(f"This request has status '{remote.status}' and cannot be downloaded.")

2272e7cf7ebac1f16f0784a7d5d52ee7.grib:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

✅ Download complete!


### Delete an Old Request

In [10]:
# If you want to remove an old request:
request_id = "49b3a61b-7310-4da3-b22f-f9e85f2a9821"  # Replace with an ID from your table

# Delete the request
remote = client.get_remote(request_id)
remote.delete()
print(f"✅ Request deleted successfully!")

✅ Request deleted successfully!


## Tips for Managing Your Data Requests

- **Sort by date**: Keep track of when you made each request
- **Look at the status**: "successful" means the data is ready to download
- **Check file formats**: Make sure you know what format your data is in before downloading
- **Keep track of request IDs**: Save important request IDs for data you might need again
- **Clean up periodically**: Delete old requests you no longer need
